<a href="https://colab.research.google.com/github/adnaksbhat/Colab_JupyterNotebooks/blob/main/ECR_All_Closed_2020_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ECR_All_Closed_2020_2024

##data

In [ ]:
import pandas as pd

df_og = pd.read_excel("/content/ECR_All_Closed_2020_2024.xlsx")


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

FileNotFoundError: [Errno 2] No such file or directory: '/content/ECR_All_Closed_2020_2024.xlsx'

In [ ]:
df_og.head(2)

NameError: name 'df_og' is not defined

In [ ]:
df = df_og[['ECR_CODE', 'SOURCE OF CHANGE', 'ECR_CREATION_DATE', 'REASON_DESC', 'CHANGE_TYPE','COSTING_PROJECTS','MATERIAL_CATEGORY','CHANGE_CATEGORY','CHANGE DESCRIPTION','IMPACTEDITEM', 'IMPACTED ITEM DESCRIPTION','FUNCTION','DIVISION','ORGANIZATION NAME','AGING','SUB_PL']]
df.head()

NameError: name 'df_og' is not defined

In [ ]:
df_FFF= df[(df['CHANGE_TYPE'] == 'FFF&P ONLY')]
df_CIS_FFF = df[(df['CHANGE_TYPE'] == 'CIS/CIS+FFF&P')]

In [ ]:
df_FFF.head(2)

In [ ]:
df_CIS_FFF.head(2)

##graphs

In [ ]:
# FUNCTION

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('FUNCTION').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.title("Function vs Count")
plt.xlabel("Count")
plt.ylabel("Function")

In [ ]:
# SOURCE OF CHANGE

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('SOURCE OF CHANGE').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.title("Source of Change vs Count")
plt.xlabel("Count")
plt.ylabel("Source of Change")

##ECR_CODE
Each ECR_CODE has multiple COSTING_PROJECTS


So, ECR_CODE is grouped along with COSTING_PROJECTS(Job No.)


Now ECR_CODE can be used as PK



In [ ]:
def group_cols(df, colA, colB):
    grouped_df = df.groupby(colA)[colB].agg(list).reset_index()
    grouped_df.rename(columns={colB: colB + '_new_col'}, inplace=True)
    df = pd.merge(df, grouped_df, on=colA, how='left')
    df.drop(colB, axis=1, inplace=True)
    return df

In [ ]:
df=group_cols(df,'ECR_CODE','COSTING_PROJECTS')

In [ ]:
# change type to string
# remove brackets
# drop duplicate so that ECR_CODE can act as PK
df['COSTING_PROJECTS_new_col']=df['COSTING_PROJECTS_new_col'].astype(str)
df['COSTING_PROJECTS_new_col'] = df['COSTING_PROJECTS_new_col'].str.replace(r'[\[\]\']', '', regex=True)


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.head(50)

In [ ]:
df.shape

In [ ]:
# Function to sort each cell
def sort_costing_projects(cell):
    # Ensure the cell is a string before processing
    if isinstance(cell, list):
        cell = ', '.join(cell)  # Convert list to string
    # Split, strip whitespace, sort, and rejoin
    sorted_items = sorted(item.strip() for item in cell.split(','))
    return ', '.join(sorted_items)


In [ ]:
# Apply the function to each cell in the specified column
df['COSTING_PROJECTS_new_col'] = df['COSTING_PROJECTS_new_col'].apply(sort_costing_projects)



In [ ]:
df.head()

In [ ]:
# Top 10 most mentioned string in df[COSTING_PROJECTS_new_col]

from collections import Counter

def top_mentioned_strings(df, column, top_n):

    all_strings = []
    for value in df[column]:
        if isinstance(value, str):  # Check if the value is a string
            strings = value.split(',')
            all_strings.extend(strings)
    string_counts = Counter(all_strings)
    top_strings = string_counts.most_common(top_n)
    return top_strings



In [ ]:

top_10_strings = top_mentioned_strings(df, 'COSTING_PROJECTS_new_col', 10)
top_10_strings

NameError: name 'df' is not defined

In [ ]:
# prompt: plot above top_10_strings

import matplotlib.pyplot as plt

# Assuming top_10_strings is a list of tuples (string, count)
strings, counts = zip(*top_10_strings)

plt.figure(figsize=(10, 6))
plt.bar(strings, counts,color='pink')
plt.xlabel("COSTING_PROJECTS")
plt.ylabel("Count")
plt.title("Top 10 Most Mentioned COSTING_PROJECTS")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

##modified df  |  made costing_projects as pk and has been grouped togather

In [ ]:
# prompt: if  df[COSTING_PROJECTS_new_col] value is same then merge those rows

def merge_rows_with_same_costing_projects(df):
    # Group by 'COSTING_PROJECTS_new_col' and aggregate other columns
    grouped = df.groupby('COSTING_PROJECTS_new_col').agg({
        'ECR_CODE': lambda x: ','.join(x.astype(str)),  # Combine ECR codes
        'CHANGE DESCRIPTION': lambda x: ','.join(x.astype(str)),
        'IMPACTEDITEM': lambda x: ','.join(x.astype(str)), #Combine IMPACTEDITEM values


    }).reset_index()

    return grouped

df_merged = merge_rows_with_same_costing_projects(df)
df_merged

In [ ]:
df_merged.shape

In [ ]:
df['ECR_CODE'].value_counts()

In [ ]:
# ECR-550831	has 2 values coz IMPACTED ITEM DESCRIPTION	is differently described

In [ ]:
'''

Method to find all rows which has more than 2 values in a col


'''

In [ ]:
# Method to find all rows which has more than 2 values in a col. example [1908315, 1703775]

def find_rows_with_multiple_values(df, colA, value):

    result_df = df[df[colA].str.split(',').str.len() > value]

    return result_df

In [ ]:
df_multiple_costingprojects=find_rows_with_multiple_values(df,'COSTING_PROJECTS_new_col',1)

In [ ]:
df_multiple_costingprojects.head()

In [ ]:
#grouping costing_projects to ECR_code



##SOURCE OF CHANGE

has 5 unique values.

So, label encode these

In [ ]:
df.head(2)

In [ ]:
# prompt: plot graph on df['SOURCE OF CHANGE']

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
df['SOURCE OF CHANGE'].value_counts().plot(kind='barh')
plt.ylabel('Source of Change')
plt.xlabel('Count')
plt.title('Distribution of Source of Change')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.show()

In [ ]:
df['SOURCE OF CHANGE'].unique()

##REASON_DESC

has unique values

So, label encode these

In [ ]:
df['REASON_DESC'].value_counts()

NameError: name 'df' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
df['REASON_DESC'].value_counts().plot(kind='barh')
plt.ylabel('REASON_DESC')
plt.xlabel('Count')
plt.title('Distribution of REASON_DESC')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.show()

In [ ]:
# prompt: group REASON_DESC and MATERIAL_CATEGORY and find top 10 MATERIAL_CATEGORY for each REASON_DESC

# Group by 'REASON_DESC' and 'MATERIAL_CATEGORY', then get the top 10 'MATERIAL_CATEGORY' for each 'REASON_DESC'
top_10_materials_for_reason = df.groupby('REASON_DESC')['MATERIAL_CATEGORY'].value_counts().groupby(level=0).head(5)

top_10_materials_for_reason

In [ ]:


import matplotlib.pyplot as plt

# Assuming top_10_materials_for_reason is already defined as in your provided code
# Iterate through each unique REASON_DESC
for reason in top_10_materials_for_reason.index.get_level_values(0).unique():
    # Get the top 10 materials for the current reason
    reason_data = top_10_materials_for_reason[reason]
    # Create the pie chart
    plt.figure(figsize=(18, 16))  # Adjust figure size as needed
    plt.pie(reason_data, autopct='%1.1f%%', startangle=90, explode=[0.05] * len(reason_data)) #Added autopct for percentage values
    plt.title(f'Reason: {reason}')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.legend(reason_data.index, loc='upper left', bbox_to_anchor=(1, 1)) #legend only
    plt.show()

In [ ]:
df['REASON_DESC'].unique()

##CHANGE_CATEGORY

eventhough it has unique values, its in mixtured form
apply multilabel binarizer

here u can ignore 'DOCUMENT' col as they cause no major impacts

In [ ]:
df['CHANGE_CATEGORY'].unique()

In [ ]:
#seperate if df['CHANGE_CATEGORY']=='DOCUMENT'

In [ ]:
df_document=df[df['CHANGE_CATEGORY']=='DOCUMENT']






df=df[df['CHANGE_CATEGORY']!='DOCUMENT']

In [ ]:
df_document.head()     #will have only documents

In [ ]:
df.head()

In [ ]:
df['CHANGE_CATEGORY']=df['CHANGE_CATEGORY'].astype(str)

In [ ]:
from collections import Counter

def count_change_category_words(text):
    words = text.lower().split(',')
    return Counter(words)

change_category_word_counts = df['CHANGE_CATEGORY'].apply(count_change_category_words)

# Combine word counts from all rows
total_change_category_word_counts = Counter()
for counts in change_category_word_counts:
    total_change_category_word_counts.update(counts)

total_change_category_word_counts

In [ ]:
def search_change_category(word):
  word = word.lower()

  return total_change_category_word_counts[word]

In [ ]:
search_change_category("move component")

In [ ]:
def find_word_in_change_category(word, change_type):
  word = word.lower()


  matching_rows = df[(df['CHANGE_CATEGORY'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','MATERIAL_CATEGORY']]
  else:
    return None

In [ ]:
find_word_in_change_category("move component", "FFF&P ONLY")

##MATERIAL_CATEGORY

has unique values

So, label encode these

In [ ]:
# prompt: for each of df[COSTING_PROJECTS] find top MATERIAL_CATEGORY value

def get_top_material_category(df):


    # Group by 'COSTING_PROJECTS' and get the mode of 'MATERIAL_CATEGORY'
    top_material_categories = df.groupby('COSTING_PROJECTS_new_col')['MATERIAL_CATEGORY'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown').reset_index()
    return top_material_categories


# Example usage (assuming your DataFrame is named 'df'):
top_categories_df = get_top_material_category(df)

top_categories_df

In [ ]:
# prompt: plot pie graph for top 10 MATERIAL_CATEGORY for above top_categories_d

import matplotlib.pyplot as plt

# Assuming 'top_categories_df' is already defined as in your provided code.
top_10_materials = top_categories_df['MATERIAL_CATEGORY'].value_counts().nlargest(5)

plt.figure(figsize=(10, 10))
plt.pie(top_10_materials, labels=top_10_materials.index, autopct='%1.1f%%', startangle=90)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title("Top 5 material categories")
plt.show()

In [ ]:
# prompt: top 5 MATERIAL_CATEGORY values

# Assuming df is your DataFrame as defined in the provided code.
top_5_materials = df['MATERIAL_CATEGORY'].value_counts().nlargest(5)
top_5_materials

In [ ]:
# prompt: plot graph for top_5_materials

import matplotlib.pyplot as plt

# Assuming top_5_materials is already defined as in your provided code.
plt.figure(figsize=(8, 6))
top_5_materials.plot(kind='bar', color='skyblue')
plt.title('Top 5 Material Categories')
plt.xlabel('Material Category')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
df['MATERIAL_CATEGORY'].value_counts()

In [ ]:
df['MATERIAL_CATEGORY']=df['MATERIAL_CATEGORY'].astype(str)

In [ ]:
from collections import Counter

def count_material_words(text):
    words = text.lower().split(',')
    return Counter(words)

material_category_word_counts = df['MATERIAL_CATEGORY'].apply(count_material_words)

# Combine word counts from all rows
total_material_category_word_counts = Counter()
for counts in material_category_word_counts:
    total_material_category_word_counts.update(counts)

total_material_category_word_counts

In [ ]:
def search_material_category(word):
  word = word.lower()

  return total_material_category_word_counts[word]

In [ ]:
search_material_category("gearbox")

NameError: name 'total_material_category_word_counts' is not defined

In [ ]:
def find_word_in_material_category(word, change_type):
  word = word.lower()


  matching_rows = df[(df['MATERIAL_CATEGORY'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','MATERIAL_CATEGORY']]
  else:
    return None

In [ ]:
find_word_in_material_category("gearbox", "FFF&P ONLY")

##CHANGE DESCRIPTION

convert to english

perform text preprocessing

extract doc if mentioned in a new col

In [ ]:
df.head(2)

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
df.head()

In [ ]:

df['CHANGE DESCRIPTION'] = df['CHANGE DESCRIPTION'].astype(str)

In [ ]:

import pandas as pd
from googletrans import Translator

# Assuming df is your DataFrame and 'CHANGE DESCRIPTION' is the column to translate.
translator = Translator()

def translate_column(df, column_name):
    # Handle potential errors during translation
    def translate_text(text):
      return translator.translate(str(text), dest='en').text  # Ensure text is a string

    df[column_name + '_translated'] = df[column_name].apply(translate_text)
    return df

In [ ]:
translate_column(df,'CHANGE DESCRIPTION')

In [ ]:
''''

# prompt: divide df into many parts. apply translate_column(df, 'CHANGE DESCRIPTION') to each and merge all df


# Divide df into smaller parts (e.g., chunks of 100 rows)
chunk_size = 100
translated_dfs = []

for i in range(0, len(df), chunk_size):
  chunk = df[i:i + chunk_size].copy()
  chunk = translate_column(chunk, 'CHANGE DESCRIPTION')
  translated_dfs.append(chunk)

# Concatenate the translated chunks back into a single DataFrame
df = pd.concat(translated_dfs, ignore_index=True)


'''

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
'''

!pip install BHDataAnalysis

'''

In [ ]:
'''

from BHDataAnalysis import featureEngineeringFile

!pip show BHDataAnalysis

'''

In [ ]:
########          ALMIGHTY PROCESS METHOD


import os
import nltk
import spacy

nlp=spacy.load('en_core_web_sm')



def completePreprocessMethod(str):



  #Tokenization
  #==================
  token_doc=nlp(str.lower())
  token_array=[]

  for token in token_doc:
    token_array.append(token.text)
    token_string=' '.join(token_array)

  print("\n After Tokenization-------->\n",token_string)


  #remove stopwords and punct and lemmatize
  #==================
  from spacy.lang.en.stop_words import STOP_WORDS

  stoppunclemma_doc=nlp(token_string)
  stoppunclemma_array=[]

  for token in stoppunclemma_doc:

    if not (token.is_stop or token.is_punct):
      stoppunclemma_array.append(token.lemma_)

  stoppunclemma_string=' '.join(stoppunclemma_array)

  print("\n After removing stopwords and punctuations and lemmatizing-------->\n",stoppunclemma_string)



  '''
  #stemming
  #==================
  words=stoppunclemma_array
  from nltk.stem import PorterStemmer
  stemmer=PorterStemmer()
  stemmArray=[]

  for i in words:
    stemmArray.append(stemmer.stem(i))

  string_stemm = ' '.join(stemmArray)
  print("\nAfter stemming-------->\n",string_stemm)

  '''


  '''
  #lemma
  #==================
  lemma_doc=nlp(string_stemm)

  lemmaArray=[]

  for i in lemma_doc:
    lemmaArray.append(i.lemma_)

  string_lemma = ' '.join(lemmaArray)
  print("\nafter lemmatization-------->\n",string_lemma)

  '''


  final_string=stoppunclemma_string

  print("Final string\n")

  return final_string



In [ ]:
def almightyPreprocessMethod(anything):
  if isinstance(anything,str):
    if anything.endswith('.txt'):
      #text file as param
      with open(anything) as f:
        text=f.readlines()

      proper_text = ' '.join(text)

      final_string=complete_preprocess(proper_text)
      return final_string


    else:
      #string as param
      final_string=completePreprocessMethod(anything)
      return final_string

  elif isinstance(anything,list):
    #list/array as param
    tokenized_string=' '.join(anything)

    final_string=completePreprocessMethod(tokenized_string)
    return final_string

  else:
    return "Invalid param passed"

In [ ]:
df['CHANGE DESCRIPTION_translated_preprocessed']=df['CHANGE DESCRIPTION_translated'].apply(lambda x: almightyPreprocessMethod(x))


In [ ]:
df.head()

In [ ]:
# new col to find docs in change description

In [ ]:
df.head(2)

In [ ]:
df['CHANGE DESCRIPTION_translated_preprocessed']=df['CHANGE DESCRIPTION_translated_preprocessed'].astype(str)

In [ ]:
# if a word which contains any numericals in df[CHANGE DESCRIPTION_translated], get ONLY that values and add it in new column 'Doc_name_from_description'. if it is only alphabets, dont add it

import re

def extract_doc_numbers(text):
    # Regular expression to find numerical sequences (potential document numbers)
    doc_numbers = re.findall(r'\w+\d[\w\d]+', text)
    return ','.join(doc_numbers) if doc_numbers else ''

df['Docs_in_CHANGE DESCRIPTION'] = df['CHANGE DESCRIPTION_translated_preprocessed'].apply(extract_doc_numbers)


In [ ]:
df.head()

In [ ]:
# Total 54 rows has no doc mentioned in Change description.




df['Docs_in_CHANGE DESCRIPTION'].eq('').sum()



In [ ]:
# from this we can see each change description is UNIQUE.  need to do cosine similarity | or analyse based on only ADD component, remove component etc

df['CHANGE DESCRIPTION_translated'].value_counts()

In [ ]:
# prompt: find count of every word in  df[CHANGE DESCRIPTION_translated_preprocessed]

from collections import Counter

def count_words(text):
    words = text.lower().split()
    return Counter(words)

word_counts = df['CHANGE DESCRIPTION_translated_preprocessed'].apply(count_words)

# Combine word counts from all rows
total_word_counts = Counter()
for counts in word_counts:
    total_word_counts.update(counts)

total_word_counts

In [ ]:
# create a method which takes in a word and returns total_word_counts value

def search_change_description(word):
  word = word.lower()

  return total_word_counts[word]

In [ ]:
search_change_description('siemens')

In [ ]:
#  a method which takes a word as input. check if it is present in df[CHANGE DESCRIPTION_translated_preprocessed], if yes, then return entire row

def find_word_in_change_description(word,change_type):
  word = word.lower()


  matching_rows = df[(df['CHANGE DESCRIPTION_translated_preprocessed'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','CHANGE DESCRIPTION_translated']]
  else:
    return None

In [ ]:

find_word_in_change_description("siemens" ,"CIS/CIS+FFF&P")


## IMPACTED ITEM DESCRIPTION

In [ ]:
df.head(2)

In [ ]:
df['IMPACTED ITEM DESCRIPTION_preprocessed']=df['IMPACTED ITEM DESCRIPTION'].apply(lambda x: almightyPreprocessMethod(x))


In [ ]:
df.head(2)

In [ ]:



from collections import Counter

def count_words(text):
    words = text.lower().split()
    return Counter(words)

impacted_item_word_counts = df['IMPACTED ITEM DESCRIPTION_preprocessed'].apply(count_words)

# Combine word counts from all rows
total_impacted_item_word_counts = Counter()
for counts in impacted_item_word_counts:
    total_impacted_item_word_counts.update(counts)

total_impacted_item_word_counts

In [ ]:
def search_impacted_item(word):
  word = word.lower()

  return total_impacted_item_word_counts[word]

In [ ]:
search_impacted_item('valve')

In [ ]:
def find_word_in_impacted_item(word, change_type):
  word = word.lower()


  matching_rows = df[(df['IMPACTED ITEM DESCRIPTION_preprocessed'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','IMPACTEDITEM','IMPACTED ITEM DESCRIPTION_preprocessed']]
  else:
    return None

In [ ]:
find_word_in_impacted_item("flux", "FFF&P ONLY")

##Save csv | final csv after preprocessing

In [ ]:
# : save above df

from google.colab import files
df.to_csv('df_final.csv')
files.download('df_final.csv')

In [ ]:
import pandas as pd

df=pd.read_csv('/content/df_final.csv')

## Search methods

In [ ]:
import pandas as pd

df2=pd.read_csv('/content/df_final.csv')

In [ ]:
# prompt: make all col in df to str

for col in df2.columns:
  df2[col] = df2[col].astype(str)

In [ ]:
df2['CHANGE DESCRIPTION']=df2['CHANGE DESCRIPTION'].astype(str)

In [ ]:
df2.info()

In [ ]:
from collections import Counter

def count_change_category_words(text):
    words = text.lower().split(',')
    return Counter(words)

change_category_word_counts = df['CHANGE_CATEGORY'].apply(count_change_category_words)

# Combine word counts from all rows
total_change_category_word_counts = Counter()
for counts in change_category_word_counts:
    total_change_category_word_counts.update(counts)

total_change_category_word_counts

In [ ]:
def search_change_category(word):
  word = word.lower()

  return total_change_category_word_counts[word]

In [ ]:
search_change_category("move component")

In [ ]:
def find_word_in_change_category(word, change_type):
  word = word.lower()


  matching_rows = df[(df['CHANGE_CATEGORY'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','MATERIAL_CATEGORY']]
  else:
    return None

In [ ]:
find_word_in_change_category("move component", "FFF&P ONLY")

In [ ]:
from collections import Counter

def count_material_words(text):
    words = text.lower().split(',')
    return Counter(words)

material_category_word_counts = df['MATERIAL_CATEGORY'].apply(count_material_words)

# Combine word counts from all rows
total_material_category_word_counts = Counter()
for counts in material_category_word_counts:
    total_material_category_word_counts.update(counts)

total_material_category_word_counts

In [ ]:
def search_material_category(word):
  word = word.lower()

  return total_material_category_word_counts[word]

In [ ]:
search_material_category("gearbox")

In [ ]:
def find_word_in_material_category(word, change_type):
  word = word.lower()


  matching_rows = df[(df['MATERIAL_CATEGORY'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','MATERIAL_CATEGORY']]
  else:
    return None

In [ ]:
find_word_in_material_category("gearbox", "FFF&P ONLY")

In [ ]:
# prompt: find count of every word in  df[CHANGE DESCRIPTION_translated_preprocessed]

from collections import Counter

def count_words(text):
    words = text.lower().split()
    return Counter(words)

word_counts = df['CHANGE DESCRIPTION_translated_preprocessed'].apply(count_words)

# Combine word counts from all rows
total_word_counts = Counter()
for counts in word_counts:
    total_word_counts.update(counts)

total_word_counts

In [ ]:
# create a method which takes in a word and returns total_word_counts value

def search_change_description(word):
  word = word.lower()

  return total_word_counts[word]

In [ ]:
search_change_description('siemens')

In [ ]:
#  a method which takes a word as input. check if it is present in df[CHANGE DESCRIPTION_translated_preprocessed], if yes, then return entire row

def find_word_in_change_description(word,change_type):
  word = word.lower()


  matching_rows = df[(df['CHANGE DESCRIPTION_translated_preprocessed'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','CHANGE DESCRIPTION_translated']]
  else:
    return None

In [ ]:

find_word_in_change_description("siemens" ,"CIS/CIS+FFF&P")


In [ ]:



from collections import Counter

def count_words(text):
    words = text.lower().split()
    return Counter(words)

impacted_item_word_counts = df['IMPACTED ITEM DESCRIPTION_preprocessed'].apply(count_words)

# Combine word counts from all rows
total_impacted_item_word_counts = Counter()
for counts in impacted_item_word_counts:
    total_impacted_item_word_counts.update(counts)

total_impacted_item_word_counts

In [ ]:
def search_impacted_item(word):
  word = word.lower()

  return total_impacted_item_word_counts[word]

In [ ]:
search_impacted_item('valve')

In [ ]:
def find_word_in_impacted_item(word, change_type):
  word = word.lower()


  matching_rows = df[(df['IMPACTED ITEM DESCRIPTION_preprocessed'].str.contains(word, case=False, na=False)) & (df['CHANGE_TYPE'] == change_type)]

  if not matching_rows.empty:
    return matching_rows[['ECR_CODE', 'COSTING_PROJECTS_new_col','CHANGE_CATEGORY','IMPACTEDITEM','IMPACTED ITEM DESCRIPTION_preprocessed']]
  else:
    return None

In [ ]:
find_word_in_impacted_item("flux", "FFF&P ONLY")

## Prepare data for training | all transformation, encoding, etc

In [ ]:
# label encode SOURCE OF CHANGE

#label encode these

from sklearn.preprocessing import LabelEncoder

le_source_of_change = LabelEncoder()
df['SOURCE OF CHANGE'] = le_source_of_change.fit_transform(df['SOURCE OF CHANGE'])

le_source_of_change_classes = dict(zip(le_source_of_change.classes_, le_source_of_change.transform(le_source_of_change.classes_)))

le_source_of_change_classes

In [ ]:
# label encode REASON DESC

le_reason_desc = LabelEncoder()
df['REASON_DESC'] = le_reason_desc.fit_transform(df['REASON_DESC'])

le_reason_desc_classes = dict(zip(le_reason_desc.classes_, le_reason_desc.transform(le_reason_desc.classes_)))

le_reason_desc_classes

In [ ]:
#multilabel binarizer for CHANGE_CATEGORY col


from sklearn.preprocessing import MultiLabelBinarizer
df['CHANGE_CATEGORY']=df['CHANGE_CATEGORY'].astype(str)
df['CHANGE_CATEGORY'] = df['CHANGE_CATEGORY'].apply(lambda x: x.split(','))


mlb = MultiLabelBinarizer()
change_category_encoded = pd.DataFrame(mlb.fit_transform(df['CHANGE_CATEGORY']),
                                       columns=mlb.classes_,
                                       index=df.index)

# Concatenate the original data with the encoded columns
df = pd.concat([df, change_category_encoded], axis=1)

# Drop the original 'CHANGE_CATEGORY' column if no longer needed
df.drop(columns=['CHANGE_CATEGORY'], inplace=True)

In [ ]:
print("ADD COMPONENT -> ", len(df[df['ADD COMPONENT']==1]))

print("CHANGE QTY -> ", len(df[df['CHANGE QTY']==1]))

print("COSTOUT/SMI -> ", len(df[df['COSTOUT/SMI']==1]))

print("DOCUMENT -> ", len(df[df['DOCUMENT']==1]))

print("MOVE COMPONENT -> ", len(df[df['MOVE COMPONENT']==1]))

print("NEW PRODUCT DEVELOPMENT (NPD) -> ", len(df[df['NEW PRODUCT DEVELOPMENT (NPD)']==1]))

print("PCB CATEGORY -> ", len(df[df['PCB CATEGORY']==1]))

print("REMOVE COMPONENT -> ", len(df[df['REMOVE COMPONENT']==1]))


In [ ]:
import matplotlib.pyplot as plt

# Data for the bar chart (replace with your actual data)
categories = ['ADD COMPONENT', 'CHANGE QTY', 'COSTOUT/SMI', 'DOCUMENT', 'MOVE COMPONENT', 'NEW PRODUCT DEVELOPMENT (NPD)', 'PCB CATEGORY', 'REMOVE COMPONENT']
counts = [len(df[df['ADD COMPONENT']==1]), len(df[df['CHANGE QTY']==1]), len(df[df['COSTOUT/SMI']==1]), len(df[df['DOCUMENT']==1]), len(df[df['MOVE COMPONENT']==1]), len(df[df['NEW PRODUCT DEVELOPMENT (NPD)']==1]), len(df[df['PCB CATEGORY']==1]), len(df[df['REMOVE COMPONENT']==1])]

# Create the bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(categories, counts, color='purple')
plt.xlabel("Change Categories")
plt.ylabel("Count")
plt.title("Count of Different Change Categories")
plt.xticks(rotation=45, ha='right')

# Add value labels on top of each bar
for bar, count in zip(bars, counts):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(count), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
#try doing multilabel binarizer or


#multilabel binarizer for MATERIAL_CATEGORY col


from sklearn.preprocessing import MultiLabelBinarizer
df['MATERIAL_CATEGORY']=df['MATERIAL_CATEGORY'].astype(str)
df['MATERIAL_CATEGORY'] = df['MATERIAL_CATEGORY'].apply(lambda x: x.split(','))


mlb = MultiLabelBinarizer()
material_category_encoded = pd.DataFrame(mlb.fit_transform(df['MATERIAL_CATEGORY']),
                                       columns=mlb.classes_,
                                       index=df.index)

# Concatenate the original data with the encoded columns
df = pd.concat([df, material_category_encoded], axis=1)




'''
#label encoder
le_material_category = LabelEncoder()
df['MATERIAL_CATEGORY'] = le_material_category.fit_transform(df['MATERIAL_CATEGORY'])

le_material_category_classes = dict(zip(le_material_category.classes_, le_material_category.transform(le_material_category.classes_)))

le_material_category_classes
'''

In [ ]:
print("AUXILIARY BASEPLATE -> ", len(df[df['AUXILIARY BASEPLATE']==1]))

print("BASEPLATE -> ", len(df[df['BASEPLATE']==1]))

print("CABLES -> ", len(df[df['CABLES']==1]))

print("CONTROL CABINET -> ", len(df[df['CONTROL CABINET']==1]))

print("COUPLING -> ", len(df[df['COUPLING']==1]))

print("COUPLING GUARD -> ", len(df[df['COUPLING GUARD']==1]))

print("DRY GAS SEAL PANEL/TREATMENT/BOOSTER -> ", len(df[df['DRY GAS SEAL PANEL/TREATMENT/BOOSTER']==1]))

print("DUCTS -> ", len(df[df['DUCTS']==1]))

print("ENCLOSURE -> ", len(df[df['ENCLOSURE']==1]))

print("EQUIPMENT/MATERIAL FULL SUPPLY -> ", len(df[df['EQUIPMENT/MATERIAL FULL SUPPLY']==1]))



print("F2F -> ", len(df[df['F2F']==1]))

print("FILTER HOUSE -> ", len(df[df['FILTER HOUSE']==1]))

print("GEARBOX -> ", len(df[df['GEARBOX']==1]))

#print("GEARBOX RAW -> ", len(df[df['GEARBOX RAW']==1]))

print("INSTRUMENTATION -> ", len(df[df['INSTRUMENTATION']==1]))

print("INTERCONNECTING PIPING AND SUPPORTS -> ", len(df[df['INTERCONNECTING PIPING AND SUPPORTS']==1]))

print("LOW VOLTAGE MOTOR -> ", len(df[df['LOW VOLTAGE MOTOR']==1]))

print("LUBE OIL CONSOLE -> ", len(df[df['LUBE OIL CONSOLE']==1]))

print("MAIN MOTOR/GENERATOR -> ", len(df[df['MAIN MOTOR/GENERATOR']==1]))

print("MCC -> ", len(df[df['MCC']==1]))



print("OTHER -> ", len(df[df['OTHER']==1]))

print("PIPINGS & SUPPORTS -> ", len(df[df['PIPINGS & SUPPORTS']==1]))

print("PUMP FOR AUXILIARIES -> ", len(df[df['PUMP FOR AUXILIARIES']==1]))

print("SHAFT LINE-> ", len(df[df['SHAFT LINE']==1]))

print("SPARE PARTS -> ", len(df[df['SPARE PARTS']==1]))

print("SPECIAL TOOLS -> ", len(df[df['SPECIAL TOOLS']==1]))

print("STEEL STRUCTURES -> ", len(df[df['STEEL STRUCTURES']==1]))

print("UCP/CONTROL SYSTEM -> ", len(df[df['UCP/CONTROL SYSTEM']==1]))

print("VALVES -> ", len(df[df['VALVES']==1]))

print("MCC -> ", len(df[df['MCC']==1]))


In [ ]:
#  enter a JobNo. and a col (add component, remove, anything)  based on this it will tell which Material category is affected along with its ECR_CODE

def get_material_categories(costing_projects_value, col):
    """
    This method takes in 'COSTING_PROJECTS_new_col' value and 'ADD COMPONENT' value
    and returns all material categories associated with the given criteria along with ECR_CODE.
    """
    # Filter the DataFrame based on the provided criteria
    filtered_df = df[(df['COSTING_PROJECTS_new_col'] == costing_projects_value) & (df[col] == 1)]

    # Extract the material categories and ECR_CODE from the filtered data
    if not filtered_df.empty:
        for index, row in filtered_df.iterrows():
            print(f"ECR_CODE: {row['ECR_CODE']}, Material Category: {row['MATERIAL_CATEGORY']}")
    else:
        print("No material categories found for given criteria")


In [ ]:
get_material_categories("315529", 'REMOVE COMPONENT')

In [ ]:
# prompt: translate df[column]  to english

def translate_column(df, column_name):
  """Translates the specified column in a DataFrame to English.

  Args:
      df: The input DataFrame.
      column_name: The name of the column to translate.

  Returns:
      The DataFrame with the translated column added.
  """
  translator = Translator()
  df[column_name + '_translated'] = df[column_name].astype(str).apply(lambda x: translator.translate(x, dest='en').text)
  return df

# Example usage (assuming your DataFrame is named 'df' and the column to translate is 'column'):
df = translate_column(df, 'DESCRIPTION')